In [ ]:
"""
- Gửi ảnh JPEG chất lượng cao khi có chuyển động
- Gửi heartbeat định kỳ khi không có chuyển động
- Thread 1: Camera capture + Motion detection
- Thread 2: Face detection 
- Thread 3: Sending 
- Main thread: Coordination và error handling
"""

import os
import time
import socket
import cv2
import numpy as np
import imagezmq
import threading
import queue
from picamera2 import Picamera2

# ========== Cấu hình hệ thống ==========
CONFIG = {
    "server_ip": os.getenv("SERVER_IP", "192.168.1.9"),
    "port": int(os.getenv("PORT", "5555")),
    "jpeg_quality_motion": 75,
    "jpeg_quality_idle": 45,
    "target_fps": 12,
    "heartbeat_interval": 0.5,
    "motion_pixel_threshold": 5000,
    "diff_threshold": 25,
    "blur_kernel": 9, 
    "frame_size": (640, 480),
    "queue_maxsize": 2,
    "thread_timeout": 0.5
}

CAMERA_NAME = socket.gethostname()
CONNECT_TO = f"tcp://{CONFIG['server_ip']}:{CONFIG['port']}"
FACE_CASCADE = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


# Cấu hình nhận diện
FR_CONFIG = {
    "thresh": 0.60,
    "detect_every_n": 2,   # detect mỗi N frame để giảm tải
    "min_face": 48,        # bỏ mặt quá nhỏ
}

# trạng thái nhận diện để chia tải theo N frame
_face_state = {"boxes": [], "labels": [], "frame_idx": 0}

# ========== Shared Data Structures ==========
frame_queue = queue.Queue(maxsize=CONFIG["queue_maxsize"])
send_queue = queue.Queue(maxsize=CONFIG["queue_maxsize"])
stop_event = threading.Event()


# ===== Nhận diện khuôn mặt =====
class HaarFaceDetector:
    def __init__(self):
        haar_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
        if not os.path.exists(haar_path):
            raise RuntimeError("Không tìm thấy haarcascade_frontalface_default.xml")
        self.det = cv2.CascadeClassifier(haar_path)

    def detect(self, gray):
        faces = self.det.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40), flags=cv2.CASCADE_SCALE_IMAGE)
        return [(int(x), int(y), int(w), int(h)) for (x, y, w, h) in faces]

try:
    _EYE_CASCADE_PATH = cv2.data.haarcascades + "haarcascade_eye_tree_eyeglasses.xml"
    EYE_CASCADE = cv2.CascadeClassifier(_EYE_CASCADE_PATH) if os.path.exists(_EYE_CASCADE_PATH) else None
except Exception:
    EYE_CASCADE = None

def align_face_by_eyes(gray_full, x, y, w, h):
    roi = gray_full[y:y+h, x:x+w]
    if EYE_CASCADE is None or roi.size == 0:
        return roi
    eyes = EYE_CASCADE.detectMultiScale(roi, scaleFactor=1.05, minNeighbors=3, minSize=(12, 12))
    if len(eyes) < 2:
        return roi
    eyes = sorted(eyes, key=lambda e: e[0])[:2]
    (ex1, ey1, ew1, eh1), (ex2, ey2, ew2, eh2) = eyes[0], eyes[1]
    p1 = (x + ex1 + ew1 // 2, y + ey1 + eh1 // 2)
    p2 = (x + ex2 + ew2 // 2, y + ey2 + eh2 // 2)
    angle = np.degrees(np.arctan2(p2[1] - p1[1], p2[0] - p1[0]))
    M = cv2.getRotationMatrix2D((x + w // 2, y + h // 2), angle, 1.0)
    aligned = cv2.warpAffine(gray_full, M, (gray_full.shape[1], gray_full.shape[0]), flags=cv2.INTER_LINEAR)
    return aligned[y:y+h, x:x+w]

# ---------- Illumination normalization (CLAHE + gamma) ----------
def preprocess_face_gray(gray_crop):
    """
    Chuẩn hóa ánh sáng cho ROI mặt: resize 96x96, CLAHE, gamma 0.8, blur nhẹ
    """
    if gray_crop is None or gray_crop.size == 0:
        return gray_crop
    g = cv2.resize(gray_crop, (96, 96), interpolation=cv2.INTER_LINEAR)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    g = clahe.apply(g)
    lut = (np.clip(((np.arange(256) / 255.0) ** 0.8) * 255.0, 0, 255)).astype(np.uint8)
    g = cv2.LUT(g, lut)
    g = cv2.GaussianBlur(g, (3, 3), 0)
    return g

def _lbp_uniform_map_table():
    table = np.zeros(256, dtype=np.uint8)
    def transitions(x):
        b = [(x >> i) & 1 for i in range(8)]
        return sum(b[i] != b[(i+1) % 8] for i in range(8))
    for val in range(256):
        t = transitions(val)
        if t <= 2:
            table[val] = bin(val).count("1")
        else:
            table[val] = 58
    return table

_LBP_TABLE = _lbp_uniform_map_table()

def lbp_u8(image):
    I = image.astype(np.uint8)
    n0 = np.roll(I, -1, axis=1)
    n1 = np.roll(np.roll(I, -1, axis=0), -1, axis=1)
    n2 = np.roll(I, -1, axis=0)
    n3 = np.roll(np.roll(I, -1, axis=0), 1, axis=1)
    n4 = np.roll(I, 1, axis=1)
    n5 = np.roll(np.roll(I, 1, axis=0), 1, axis=1)
    n6 = np.roll(I, 1, axis=0)
    n7 = np.roll(np.roll(I, 1, axis=0), -1, axis=1)
    c = I
    code = ((n0 >= c).astype(np.uint8) << 0) | ((n1 >= c).astype(np.uint8) << 1) | ((n2 >= c).astype(np.uint8) << 2) | ((n3 >= c).astype(np.uint8) << 3) | ((n4 >= c).astype(np.uint8) << 4) | ((n5 >= c).astype(np.uint8) << 5) | ((n6 >= c).astype(np.uint8) << 6) | ((n7 >= c).astype(np.uint8) << 7)
    return _LBP_TABLE[code]

def lbp_grid_hist(gray_crop, grid=(6,6)):
    roi = cv2.resize(gray_crop, (96, 96), interpolation=cv2.INTER_LINEAR)
    lbp = lbp_u8(roi)
    gh, gw = grid
    cell_h, cell_w = 96 // gh, 96 // gw
    feats = []
    for gy in range(gh):
        for gx in range(gw):
            cell = lbp[gy*cell_h:(gy+1)*cell_h, gx*cell_w:(gx+1)*cell_w]
            hist, _ = np.histogram(cell, bins=60, range=(0,60), density=False)
            feats.append(hist)
    feat = np.concatenate(feats).astype(np.float32)
    norm = np.linalg.norm(feat) + 1e-9
    return feat / norm

def chi2_distance(a, b, eps=1e-8):
    a = a.astype(np.float32); b = b.astype(np.float32)
    return float(0.5 * np.sum(((a - b) ** 2) / (a + b + eps)))

def recognize_hist(emb, db_emb, thresh=0.60, margin=0.03):
    best_name = "unknown"; best = 1e9; second = 1e9
    for name, vecs in db_emb.items():
        for v in vecs:
            d = chi2_distance(emb, np.asarray(v, dtype=np.float32))
            if d < best:
                second = best; best = d; best_name = name
            elif d < second:
                second = d
    if best <= thresh and (second - best) >= margin:
        return best_name, best, second
    return "unknown", best, second

class FaceDB:
    def __init__(self, db_dir="faces_db"):
        self.db_dir = db_dir
        os.makedirs(db_dir, exist_ok=True)
        self.emb = {}
        self._load()
    @property
    def json_path(self):
        return os.path.join(self.db_dir, "embeddings.json")
    def _load(self):
        if os.path.exists(self.json_path):
            import json
            with open(self.json_path, "r") as f:
                self.emb = json.load(f)
        else:
            self.emb = {}
    def list_identities(self):
        return sorted(list(self.emb.keys()))



# ========== Thread 1: Camera Capture và Motion Detection ==========
def camera_thread():
    """Thread chuyên capture camera và detect motion"""
    print("[CAMERA] Starting camera thread...")
    
    cam = init_camera()
    prev_gray = None
    last_frame_time = time.time()
    frame_interval = 1.0 / CONFIG["target_fps"]
    
    while not stop_event.is_set():
        try:
            # Capture frame
            frame = cam.capture_array()
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
            if CONFIG["blur_kernel"] > 1:
                gray = cv2.GaussianBlur(gray, (CONFIG["blur_kernel"], CONFIG["blur_kernel"]), 0)
            
            # Giới hạn FPS
            now = time.time()
            elapsed = now - last_frame_time
            if elapsed < frame_interval:
                time.sleep(frame_interval - elapsed)
            last_frame_time = time.time()
            
            # Detect motion
            motion, prev_gray, score = detect_motion(prev_gray, gray, CONFIG)
            
            # Tạo frame data để gửi
            frame_data = {
                'frame_rgb': frame_rgb,
                'motion': motion,
                'timestamp': now,
                'score': score
            }
            
            # Đưa vào queue (non-blocking)
            try:
                # Giữ frame mới nhất: nếu đầy thì drop oldest trước khi put
                if frame_queue.full():
                    try:
                        frame_queue.get_nowait()
                        frame_queue.task_done()
                    except queue.Empty:
                        pass
                frame_queue.put_nowait(frame_data)
            except queue.Full:
                pass
                
        except Exception as e:
            print(f"[CAMERA ERROR] {e}")
            time.sleep(0.1)
    
    cam.stop()
    print("[CAMERA] Camera thread stopped.")

# ========== Thread 2: Face Detection ==========
def face_detection_thread():
    """Thread chuyên detect face và đưa vào send queue"""
    print("[FACE] Starting face detection thread...")
    
    while not stop_event.is_set():
        try:
            # Lấy frame từ camera thread
            frame_data = frame_queue.get(timeout=CONFIG["thread_timeout"])
            
            # Nhận diện & vẽ nhãn (port từ face_recognition2.py)
            frame_with_faces = annotate_and_label(frame_data['frame_rgb'].copy())
            
            # Tạo send data
            send_data = {
                'frame_rgb': frame_with_faces,
                'motion': frame_data['motion'],
                'timestamp': frame_data['timestamp'],
                'score': frame_data['score']
            }
            
            # Đưa vào send queue
            try:
                # Giữ frame mới nhất cho network: drop oldest nếu đầy
                if send_queue.full():
                    try:
                        send_queue.get_nowait()
                        send_queue.task_done()
                    except queue.Empty:
                        pass
                send_queue.put_nowait(send_data)
            except queue.Full:
                pass
                
            frame_queue.task_done()
            
        except queue.Empty:
            continue
        except Exception as e:
            print(f"[FACE ERROR] {e}")
            time.sleep(0.1)
    
    print("[FACE] Face detection thread stopped.")
# ========== Thread 3: Network Sending ==========
def network_thread():
    """Thread chuyên gửi ảnh qua network"""
    print("[NETWORK] Starting network thread...")
    
    sender = None
    backoff = 1.0
    last_heartbeat = time.time()
    
    while not stop_event.is_set():
        try:
            # Lấy frame từ face detection thread
            send_data = send_queue.get(timeout=CONFIG["thread_timeout"])
            
            motion = send_data['motion']
            now = send_data['timestamp']
            
            # Quyết định có gửi hay không
            should_send = motion or (now - last_heartbeat) >= CONFIG["heartbeat_interval"]
            
            if should_send:
                # Kết nối nếu chưa có
                if sender is None:
                    try:
                        sender = imagezmq.ImageSender(connect_to=CONNECT_TO, REQ_REP=True)
                        print("[INFO] Connected to server.")
                        backoff = 1.0
                    except Exception as e:
                        print(f"[ERROR] Cannot connect: {e}. Retrying in {backoff:.1f}s")
                        time.sleep(backoff)
                        backoff = min(10.0, backoff * 1.5)
                        continue
                
                # Gửi frame
                try:
                    tag = "motion" if motion else "heartbeat"
                    quality = CONFIG["jpeg_quality_motion"] if motion else CONFIG["jpeg_quality_idle"]
                    send_frame(sender, send_data['frame_rgb'], tag, quality)
                    last_heartbeat = now
                except Exception as e:
                    print(f"[NETWORK ERROR] Failed to send frame: {e}")
                    sender = None
                    time.sleep(backoff)
                    backoff = min(10.0, backoff * 1.5)
            
            send_queue.task_done()
            
        except queue.Empty:
            continue
        except Exception as e:
            print(f"[NETWORK ERROR] {e}")
            time.sleep(0.1)
    
    print("[NETWORK] Network thread stopped.")

# ========== Khởi tạo camera ==========
def init_camera():
    cam = Picamera2()
    cam_config = cam.create_preview_configuration({"size": CONFIG["frame_size"]})
    cam.configure(cam_config)
    cam.start()
    return cam

# ========== Phát hiện chuyển động ==========
def detect_motion(prev_gray, curr_gray, config):
    if prev_gray is None:
        return False, curr_gray, 0

    diff = cv2.absdiff(prev_gray, curr_gray)
    _, thresh = cv2.threshold(diff, config["diff_threshold"], 255, cv2.THRESH_BINARY)
    motion_pixels = int(np.sum(thresh) / 255)
    motion_detected = motion_pixels > config["motion_pixel_threshold"]
    return motion_detected, curr_gray, motion_pixels


# ========== Gửi ảnh qua ImageZMQ ==========
def send_frame(sender, frame, tag, quality):
    cam_id = f"{CAMERA_NAME}:{tag}"
    ok, jpg = cv2.imencode(".jpg", frame, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
    if ok:
        sender.send_jpg(cam_id, jpg)

# ========== Phát hiện khuôn mặt ==========
def annotate_and_label(frame_rgb):
    # chia tải theo N frame
    st = _face_state
    idx = st["frame_idx"]
    gray = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    if idx % FR_CONFIG["detect_every_n"] == 0:
        faces = FR_DETECTOR.detect(gray)
        labels = []
        identities = FR_DB.list_identities()
        for (x, y, w, h) in faces:
            if min(w, h) < FR_CONFIG["min_face"]:
                labels.append(("unknown", 1.0)); continue
            mx = int(0.10*w); my = int(0.10*h)
            xs = max(0, x-mx); ys = max(0, y-my)
            xe = min(frame_rgb.shape[1]-1, x+w+mx); ye = min(frame_rgb.shape[0]-1, y+h+my)
            aligned = align_face_by_eyes(gray, xs, ys, xe-xs, ye-ys)
            if aligned.size == 0:
                labels.append(("unknown", 1.0)); continue
            proc = preprocess_face_gray(aligned)
            emb = lbp_grid_hist(proc, grid=(6,6))
            name, dist, dist2 = recognize_hist(emb, FR_DB.emb, thresh=FR_CONFIG["thresh"], margin=0.03)
            labels.append((name, dist))
        st["boxes"] = faces
        st["labels"] = labels
    # vẽ
    for (x,y,w,h), (name, dist) in zip(st["boxes"], st["labels"]):
        color = (0,200,0) if name != "unknown" else (0,0,255)
        cv2.rectangle(frame_rgb, (x,y), (x+w, y+h), color, 2)
        txt = f"{name} ({dist:.2f})" if name != "unknown" else "unknown"
        cv2.putText(frame_rgb, txt, (x, max(0, y-8)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)
    st["frame_idx"] = idx + 1
    return frame_rgb

# DB và detector cho nhận diện
FR_DETECTOR = HaarFaceDetector()
FR_DB = FaceDB()

# ========== Main Coordination ==========
def main():
    """Main function để khởi động và quản lý các threads"""
    print(f"[MAIN] Starting multi-threaded client...")
    print(f"[MAIN] Connecting to server at {CONNECT_TO}")
    
    # Khởi động các threads
    threads = []
    
    try:
        # ============ Thread 1: Camera capture và motion detection ============
        cam_thread = threading.Thread(target=camera_thread, name="CameraThread")
        cam_thread.daemon = True
        cam_thread.start()
        threads.append(cam_thread)
        
        # ============ Thread 2: Face detection =============
        face_thread = threading.Thread(target=face_detection_thread, name="FaceThread")
        face_thread.daemon = True
        face_thread.start()
        threads.append(face_thread)
        
        # ============ Thread 3: Network sending =============
        net_thread = threading.Thread(target=network_thread, name="NetworkThread")
        net_thread.daemon = True
        net_thread.start()
        threads.append(net_thread)
        
        print("[MAIN] All threads started successfully!")
        
        # Main loop để monitor và handle keyboard interrupt
        while True:
            time.sleep(1)
            
            # Kiểm tra thread health
            alive_threads = [t for t in threads if t.is_alive()]
            if len(alive_threads) < len(threads):
                print(f"[MAIN] Warning: {len(threads) - len(alive_threads)} threads died!")
                break
                
    except KeyboardInterrupt:
        print("[MAIN] Interrupted by user. Shutting down...")
    except Exception as e:
        print(f"[MAIN ERROR] {e}")
    finally:
        print("[MAIN] Stopping all threads...")
        stop_event.set()
        
        # Đợi threads kết thúc
        for thread in threads:
            thread.join(timeout=5)
            if thread.is_alive():
                print(f"[MAIN] Warning: {thread.name} did not stop gracefully")
        

if __name__ == "__main__":
    main()


In [ ]:
# """
# - Gửi ảnh JPEG chất lượng cao khi có chuyển động
# - Gửi heartbeat định kỳ khi không có chuyển động
# - Thread 1: Camera capture + Motion detection
# - Thread 2: Face detection 
# - Thread 3: Sending 
# - Main thread: Coordination và error handling
# """

# import os
# import time
# import socket
# import cv2
# import numpy as np
# import imagezmq
# import threading
# import queue
# from picamera2 import Picamera2

# # ========== Cấu hình hệ thống ==========
# CONFIG = {
#     "server_ip": os.getenv("SERVER_IP", "172.20.10.12"),
#     "port": int(os.getenv("PORT", "5555")),
#     "jpeg_quality_motion": 75,
#     "jpeg_quality_idle": 45,
#     "target_fps": 12,
#     "heartbeat_interval": 0.5,
#     "motion_pixel_threshold": 5000,
#     "diff_threshold": 25,
#     "blur_kernel": 9, 
#     "frame_size": (640, 480),
#     "queue_maxsize": 2,
#     "thread_timeout": 0.5
# }

# CAMERA_NAME = socket.gethostname()
# CONNECT_TO = f"tcp://{CONFIG['server_ip']}:{CONFIG['port']}"
# FACE_CASCADE = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# # ========== Shared Data Structures ==========
# frame_queue = queue.Queue(maxsize=CONFIG["queue_maxsize"])
# send_queue = queue.Queue(maxsize=CONFIG["queue_maxsize"])
# stop_event = threading.Event()

# # ========== Thread 1: Camera Capture và Motion Detection ==========
# def camera_thread():
#     """Thread chuyên capture camera và detect motion"""
#     print("[CAMERA] Starting camera thread...")
    
#     cam = init_camera()
#     prev_gray = None
#     last_frame_time = time.time()
#     frame_interval = 1.0 / CONFIG["target_fps"]
    
#     while not stop_event.is_set():
#         try:
#             # Capture frame
#             frame = cam.capture_array()
#             frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            
#             if CONFIG["blur_kernel"] > 1:
#                 gray = cv2.GaussianBlur(gray, (CONFIG["blur_kernel"], CONFIG["blur_kernel"]), 0)
            
#             # Giới hạn FPS
#             now = time.time()
#             elapsed = now - last_frame_time
#             if elapsed < frame_interval:
#                 time.sleep(frame_interval - elapsed)
#             last_frame_time = time.time()
            
#             # Detect motion
#             motion, prev_gray, score = detect_motion(prev_gray, gray, CONFIG)
            
#             # Tạo frame data để gửi
#             frame_data = {
#                 'frame_rgb': frame_rgb,
#                 'motion': motion,
#                 'timestamp': now,
#                 'score': score
#             }
            
#             # Đưa vào queue (non-blocking)
#             try:
#                 # Giữ frame mới nhất: nếu đầy thì drop oldest trước khi put
#                 if frame_queue.full():
#                     try:
#                         frame_queue.get_nowait()
#                         frame_queue.task_done()
#                     except queue.Empty:
#                         pass
#                 frame_queue.put_nowait(frame_data)
#             except queue.Full:
#                 pass
                
#         except Exception as e:
#             print(f"[CAMERA ERROR] {e}")
#             time.sleep(0.1)
    
#     cam.stop()
#     print("[CAMERA] Camera thread stopped.")

# # ========== Thread 2: Face Detection ==========
# def face_detection_thread():
#     """Thread chuyên detect face và đưa vào send queue"""
#     print("[FACE] Starting face detection thread...")
    
#     while not stop_event.is_set():
#         try:
#             # Lấy frame từ camera thread
#             frame_data = frame_queue.get(timeout=CONFIG["thread_timeout"])
            
#             # Detect faces
#             frame_with_faces = detect_faces(frame_data['frame_rgb'].copy())
            
#             # Tạo send data
#             send_data = {
#                 'frame_rgb': frame_with_faces,
#                 'motion': frame_data['motion'],
#                 'timestamp': frame_data['timestamp'],
#                 'score': frame_data['score']
#             }
            
#             # Đưa vào send queue
#             try:
#                 # Giữ frame mới nhất cho network: drop oldest nếu đầy
#                 if send_queue.full():
#                     try:
#                         send_queue.get_nowait()
#                         send_queue.task_done()
#                     except queue.Empty:
#                         pass
#                 send_queue.put_nowait(send_data)
#             except queue.Full:
#                 pass
                
#             frame_queue.task_done()
            
#         except queue.Empty:
#             continue
#         except Exception as e:
#             print(f"[FACE ERROR] {e}")
#             time.sleep(0.1)
    
#     print("[FACE] Face detection thread stopped.")
# # ========== Thread 3: Network Sending ==========
# def network_thread():
#     """Thread chuyên gửi ảnh qua network"""
#     print("[NETWORK] Starting network thread...")
    
#     sender = None
#     backoff = 1.0
#     last_heartbeat = time.time()
    
#     while not stop_event.is_set():
#         try:
#             # Lấy frame từ face detection thread
#             send_data = send_queue.get(timeout=CONFIG["thread_timeout"])
            
#             motion = send_data['motion']
#             now = send_data['timestamp']
            
#             # Quyết định có gửi hay không
#             should_send = motion or (now - last_heartbeat) >= CONFIG["heartbeat_interval"]
            
#             if should_send:
#                 # Kết nối nếu chưa có
#                 if sender is None:
#                     try:
#                         sender = imagezmq.ImageSender(connect_to=CONNECT_TO, REQ_REP=True)
#                         print("[INFO] Connected to server.")
#                         backoff = 1.0
#                     except Exception as e:
#                         print(f"[ERROR] Cannot connect: {e}. Retrying in {backoff:.1f}s")
#                         time.sleep(backoff)
#                         backoff = min(10.0, backoff * 1.5)
#                         continue
                
#                 # Gửi frame
#                 try:
#                     tag = "motion" if motion else "heartbeat"
#                     quality = CONFIG["jpeg_quality_motion"] if motion else CONFIG["jpeg_quality_idle"]
#                     send_frame(sender, send_data['frame_rgb'], tag, quality)
#                     last_heartbeat = now
#                 except Exception as e:
#                     print(f"[NETWORK ERROR] Failed to send frame: {e}")
#                     sender = None
#                     time.sleep(backoff)
#                     backoff = min(10.0, backoff * 1.5)
            
#             send_queue.task_done()
            
#         except queue.Empty:
#             continue
#         except Exception as e:
#             print(f"[NETWORK ERROR] {e}")
#             time.sleep(0.1)
    
#     print("[NETWORK] Network thread stopped.")

# # ========== Khởi tạo camera ==========
# def init_camera():
#     cam = Picamera2()
#     cam_config = cam.create_preview_configuration({"size": CONFIG["frame_size"]})
#     cam.configure(cam_config)
#     cam.start()
#     return cam

# # ========== Phát hiện chuyển động ==========
# def detect_motion(prev_gray, curr_gray, config):
#     if prev_gray is None:
#         return False, curr_gray, 0

#     diff = cv2.absdiff(prev_gray, curr_gray)
#     _, thresh = cv2.threshold(diff, config["diff_threshold"], 255, cv2.THRESH_BINARY)
#     motion_pixels = int(np.sum(thresh) / 255)
#     motion_detected = motion_pixels > config["motion_pixel_threshold"]
#     return motion_detected, curr_gray, motion_pixels


# # ========== Gửi ảnh qua ImageZMQ ==========
# def send_frame(sender, frame, tag, quality):
#     cam_id = f"{CAMERA_NAME}:{tag}"
#     ok, jpg = cv2.imencode(".jpg", frame, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
#     if ok:
#         sender.send_jpg(cam_id, jpg)

# # ========== Phát hiện khuôn mặt ==========
# def detect_faces(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     faces = FACE_CASCADE.detectMultiScale(gray, 1.3, 5)
#     for (x, y, w, h) in faces:
#         cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
#     return img


# # ========== Main Coordination ==========
# def main():
#     """Main function để khởi động và quản lý các threads"""
#     print(f"[MAIN] Starting multi-threaded client...")
#     print(f"[MAIN] Connecting to server at {CONNECT_TO}")
    
#     # Khởi động các threads
#     threads = []
    
#     try:
#         # ============ Thread 1: Camera capture và motion detection ============
#         cam_thread = threading.Thread(target=camera_thread, name="CameraThread")
#         cam_thread.daemon = True
#         cam_thread.start()
#         threads.append(cam_thread)
        
#         # ============ Thread 2: Face detection =============
#         face_thread = threading.Thread(target=face_detection_thread, name="FaceThread")
#         face_thread.daemon = True
#         face_thread.start()
#         threads.append(face_thread)
        
#         # ============ Thread 3: Network sending =============
#         net_thread = threading.Thread(target=network_thread, name="NetworkThread")
#         net_thread.daemon = True
#         net_thread.start()
#         threads.append(net_thread)
        
#         print("[MAIN] All threads started successfully!")
        
#         # Main loop để monitor và handle keyboard interrupt
#         while True:
#             time.sleep(1)
            
#             # Kiểm tra thread health
#             alive_threads = [t for t in threads if t.is_alive()]
#             if len(alive_threads) < len(threads):
#                 print(f"[MAIN] Warning: {len(threads) - len(alive_threads)} threads died!")
#                 break
                
#     except KeyboardInterrupt:
#         print("[MAIN] Interrupted by user. Shutting down...")
#     except Exception as e:
#         print(f"[MAIN ERROR] {e}")
#     finally:
#         print("[MAIN] Stopping all threads...")
#         stop_event.set()
        
#         # Đợi threads kết thúc
#         for thread in threads:
#             thread.join(timeout=5)
#             if thread.is_alive():
#                 print(f"[MAIN] Warning: {thread.name} did not stop gracefully")
        

# if __name__ == "__main__":
#     main()


In [ ]:
#Phát hiện khuôn mặt
# """
# - Gửi ảnh JPEG chất lượng cao khi có chuyển động
# - Gửi heartbeat định kỳ khi không có chuyển động
# - Giới hạn FPS để tiết kiệm tài nguyên
# - Tự động reconnect nếu mất kết nối
# """

# import os
# import time
# import socket
# import cv2
# import numpy as np
# import imagezmq
# from picamera2 import Picamera2

# # ========== Cấu hình hệ thống ==========
# CONFIG = {
#     "server_ip": os.getenv("SERVER_IP", "172.20.10.12"),
#     "port": int(os.getenv("PORT", "5555")),
#     "jpeg_quality_motion": 85,
#     "jpeg_quality_idle": 50,
#     "target_fps": 10,
#     # "heartbeat_interval": 0.6,
#     "motion_pixel_threshold": 5000,
#     "diff_threshold": 25,
#     "blur_kernel": 19, # int(frame_size[0] * 0.03)
#     "frame_size": (640, 480)
# }

# CAMERA_NAME = socket.gethostname()
# CONNECT_TO = f"tcp://{CONFIG['server_ip']}:{CONFIG['port']}"
# FACE_CASCADE = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# # ========== Khởi tạo camera ==========
# def init_camera():
#     cam = Picamera2()
#     cam_config = cam.create_preview_configuration({"size": CONFIG["frame_size"]})
#     cam.configure(cam_config)
#     cam.start()
#     return cam

# # ========== Phát hiện chuyển động ==========
# def detect_motion(prev_gray, curr_gray, config):
#     if prev_gray is None:
#         return False, curr_gray, 0

#     diff = cv2.absdiff(prev_gray, curr_gray)
#     _, thresh = cv2.threshold(diff, config["diff_threshold"], 255, cv2.THRESH_BINARY)
#     motion_pixels = int(np.sum(thresh) / 255)
#     motion_detected = motion_pixels > config["motion_pixel_threshold"]
#     return motion_detected, curr_gray, motion_pixels

# # ========== Gửi ảnh qua ImageZMQ ==========
# def send_frame(sender, frame, tag, quality):
#     cam_id = f"{CAMERA_NAME}:{tag}"
#     ok, jpg = cv2.imencode(".jpg", frame, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
#     if ok:
#         sender.send_jpg(cam_id, jpg)

# # ========== Phát hiện khuôn mặt ==========
# def detect_faces(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     faces = FACE_CASCADE.detectMultiScale(gray, 1.3, 5)
#     for (x, y, w, h) in faces:
#         cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
#     return img
# # ========== Main ==========
# def main():
#     print(f"[INFO] Connecting to server at {CONNECT_TO}")
#     sender = None
#     backoff = 1.0

#     cam = init_camera()
#     prev_gray = None
#     last_heartbeat = time.time()
#     last_frame_time = time.time()
#     frame_interval = 1.0 / CONFIG["target_fps"]

#     while True:
#         try:
#             frame = cam.capture_array()
#             frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#             if CONFIG["blur_kernel"] > 1:
#                 gray = cv2.GaussianBlur(gray, (CONFIG["blur_kernel"], CONFIG["blur_kernel"]), 0)

#             # Giới hạn FPS
#             now = time.time()
#             elapsed = now - last_frame_time
#             if elapsed < frame_interval:
#                 time.sleep(frame_interval - elapsed)
#             last_frame_time = time.time()

#             # Kiểm tra chuyển động
#             motion, prev_gray, score = detect_motion(prev_gray, gray, CONFIG)
#             tag = "motion" if motion else "heartbeat"
#             quality = CONFIG["jpeg_quality_motion"] if motion else CONFIG["jpeg_quality_idle"]

#             # Gửi ảnh nếu có chuyển động hoặc đến thời điểm heartbeat
#             # if motion or (now - last_heartbeat) >= CONFIG["heartbeat_interval"]:
#             if motion:
#                 if sender is None:
#                     try:
#                         sender = imagezmq.ImageSender(connect_to=CONNECT_TO, REQ_REP=True)
#                         print("[INFO] Connected to server.")
#                         backoff = 1.0
#                     except Exception as e:
#                         print(f"[ERROR] Cannot connect: {e}. Retrying in {backoff:.1f}s")
#                         time.sleep(backoff)
#                         backoff = min(10.0, backoff * 1.5)
#                         continue

#                 try:
#                     frame_rgb = detect_faces(frame_rgb)
#                     send_frame(sender, frame_rgb, tag, quality)
#                     last_heartbeat = now
#                 except Exception as e:
#                     print(f"[ERROR] Failed to send frame: {e}")
#                     sender = None
#                     time.sleep(backoff)
#                     backoff = min(10.0, backoff * 1.5)

#         except KeyboardInterrupt:
#             print("[INFO] Interrupted by user. Exiting...")
#             break
#         except Exception as e:
#             print(f"[ERROR] Unexpected error: {e}")
#             time.sleep(1)

# if __name__ == "__main__":
#     main()


In [ ]:
# """
# - Gửi ảnh JPEG chất lượng cao khi có chuyển động
# - Gửi heartbeat định kỳ khi không có chuyển động
# - Giới hạn FPS để tiết kiệm tài nguyên
# - Tự động reconnect nếu mất kết nối
# """

# import os
# import time
# import socket
# import cv2
# import numpy as np
# import imagezmq
# from picamera2 import Picamera2

# # ========== Cấu hình hệ thống ==========
# CONFIG = {
#     "server_ip": os.getenv("SERVER_IP", "172.20.10.12"),
#     "port": int(os.getenv("PORT", "5555")),
#     "jpeg_quality_motion": 85,
#     "jpeg_quality_idle": 70,
#     "target_fps": 10,
#     "heartbeat_interval": 1.0,
#     "motion_pixel_threshold": 5000,
#     "diff_threshold": 25,
#     "blur_kernel": 19, # int(frame_size[0] * 0.03)
#     "frame_size": (640, 480)
# }

# CAMERA_NAME = socket.gethostname()
# CONNECT_TO = f"tcp://{CONFIG['server_ip']}:{CONFIG['port']}"

# # ========== Khởi tạo camera ==========
# def init_camera():
#     cam = Picamera2()
#     cam_config = cam.create_preview_configuration({"size": CONFIG["frame_size"]})
#     cam.configure(cam_config)
#     cam.start()
#     return cam

# # ========== Phát hiện chuyển động ==========
# def detect_motion(prev_gray, curr_gray, config):
#     if prev_gray is None:
#         return False, curr_gray, 0

#     diff = cv2.absdiff(prev_gray, curr_gray)
#     _, thresh = cv2.threshold(diff, config["diff_threshold"], 255, cv2.THRESH_BINARY)
#     motion_pixels = int(np.sum(thresh) / 255)
#     motion_detected = motion_pixels > config["motion_pixel_threshold"]
#     return motion_detected, curr_gray, motion_pixels

# # ========== Gửi ảnh qua ImageZMQ ==========
# def send_frame(sender, frame, tag, quality):
#     cam_id = f"{CAMERA_NAME}:{tag}"
#     ok, jpg = cv2.imencode(".jpg", frame, [int(cv2.IMWRITE_JPEG_QUALITY), quality])
#     if ok:
#         sender.send_jpg(cam_id, jpg)

# # ========== Vòng lặp chính ==========
# def main():
#     print(f"[INFO] Connecting to server at {CONNECT_TO}")
#     sender = None
#     backoff = 1.0

#     cam = init_camera()
#     prev_gray = None
#     last_heartbeat = time.time()
#     last_frame_time = time.time()
#     frame_interval = 1.0 / CONFIG["target_fps"]

#     while True:
#         try:
#             frame = cam.capture_array()
#             frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#             if CONFIG["blur_kernel"] > 1:
#                 gray = cv2.GaussianBlur(gray, (CONFIG["blur_kernel"], CONFIG["blur_kernel"]), 0)

#             # Giới hạn FPS
#             now = time.time()
#             elapsed = now - last_frame_time
#             if elapsed < frame_interval:
#                 time.sleep(frame_interval - elapsed)
#             last_frame_time = time.time()

#             # Kiểm tra chuyển động
#             motion, prev_gray, score = detect_motion(prev_gray, gray, CONFIG)
#             tag = "motion" if motion else "heartbeat"
#             quality = CONFIG["jpeg_quality_motion"] if motion else CONFIG["jpeg_quality_idle"]

#             # Gửi ảnh nếu có chuyển động hoặc đến thời điểm heartbeat
#             if motion or (now - last_heartbeat) >= CONFIG["heartbeat_interval"]:
#                 if sender is None:
#                     try:
#                         sender = imagezmq.ImageSender(connect_to=CONNECT_TO, REQ_REP=True)
#                         print("[INFO] Connected to server.")
#                         backoff = 1.0
#                     except Exception as e:
#                         print(f"[ERROR] Cannot connect: {e}. Retrying in {backoff:.1f}s")
#                         time.sleep(backoff)
#                         backoff = min(10.0, backoff * 1.5)
#                         continue

#                 try:
#                     send_frame(sender, frame_rgb, tag, quality)
#                     last_heartbeat = now
#                 except Exception as e:
#                     print(f"[ERROR] Failed to send frame: {e}")
#                     sender = None
#                     time.sleep(backoff)
#                     backoff = min(10.0, backoff * 1.5)

#         except KeyboardInterrupt:
#             print("[INFO] Interrupted by user. Exiting...")
#             break
#         except Exception as e:
#             print(f"[ERROR] Unexpected error: {e}")
#             time.sleep(1)

# if __name__ == "__main__":
#     main()
